## 2.2. Komponenty biblioteki LangChain

### instalacja

In [ ]:
!pip install -q python-dotenv langchain langchain-openai langchain-community langchain-text-splitters faiss-cpu

### konfiguracja .env i modelu

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj zwięźle."),
    ("user", "Streszcz w 1 zdaniu: {tekst}")
])

chain = prompt | llm | StrOutputParser()  # LCEL: prompt → model → parser
wynik = chain.invoke({"tekst": "LangChain ułatwia budowę aplikacji LLM, dostarczając klocki do promptów, pamięci, narzędzi i RAG."})
print(wynik)


### Prompty, Parsowanie, LCEL / Chains
PromptTemplate + LLM + StrOutputParser (LCEL)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj zwięźle."),
    ("user", "Streszcz w 1 zdaniu: {tekst}")
])

chain = prompt | llm | StrOutputParser()  # LCEL: prompt → model → parser
wynik = chain.invoke({"tekst": "LangChain ułatwia budowę aplikacji LLM, dostarczając klocki do promptów, pamięci, narzędzi i RAG."})
print(wynik)


### Tools (narzędzia, które może wywołać model)
prosty tool (kalkulator) + agent ReAct

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: float, b: float) -> float:
    """Zwraca sumę a+b."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Zwraca iloczyn a*b."""
    return a * b

tools = [add, multiply]

from langchain import agents
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

react_prompt = PromptTemplate.from_template(
"""Jesteś pomocnym asystentem. Masz dostęp do narzędzi.
Używaj ich tylko gdy potrzebne. Odpowiadaj po polsku.

Pytanie: {input}
{agent_scratchpad}"""
)

agent = create_react_agent(llm, tools, react_prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = executor.invoke({"input": "Policz (12 + 7) * 3 i podaj wynik."})
print("\nWynik końcowy:", response["output"])


### Memory (historia rozmowy / stan)
RunnableWithMessageHistory (pamięć czatu)

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Prowadzisz przyjazną rozmowę i pamiętasz kontekst."),
    ("user", "{input}")
])

chat_chain = chat_prompt | llm | StrOutputParser()

store = {}  # prosta “baza” historii po session_id

def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_memory = RunnableWithMessageHistory(
    chat_chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history",
)

sid = "demo-session-1"
print(chain_with_memory.invoke({"input": "Cześć! Mam na imię Michał."},
                               config={"configurable": {"session_id": sid}}))
print(chain_with_memory.invoke({"input": "Jak mam na imię?"},
                               config={"configurable": {"session_id": sid}}))
